In [9]:
# Visualize MNIST dataset in Visdom
# Train MNIST
# Plot loss curve with Visdom

In [ ]:
# First, open Terminal and type:

# python -m visdom.server

# Then, open browser and go to http://localhost:8097/

In [10]:
import torch

In [11]:
from torchvision.datasets import MNIST

In [25]:
from torchvision import transforms

In [14]:
# TODO: talk more about what these compose options do
transf = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (1.0, ))])

In [15]:
train_set = MNIST(root='./data', train=True, transform=transf, download=False) # load training dataset

In [22]:
train_set.train_data.shape # see dataset size

torch.Size([60000, 28, 28])

In [28]:
train_set.train_data[0]

tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,

In [30]:
# To visualize the samples in dataset, let's use Visdom
from visdom import Visdom
vis = Visdom()

In [36]:
vis.image(train_set.train_data[0]) # See image in Visdom page

'window_362979aee8cc7a'

In [37]:
train_set.train_labels[0]

tensor(5)

In [38]:
test_set = MNIST(root='./data', train=False, transform=transf, download=False) # load training dataset

In [50]:
from torchvision.utils import make_grid # Combine images into a nice-looking grid layout

In [56]:
# first 10 samples combined together
# MNIST dataset doesn't have channel dimension, so we use unsqueeze to manually add it
print(train_set.train_data.shape)
print(train_set.train_data.unsqueeze(1).shape)
combined_image = make_grid(train_set.train_data[:10].unsqueeze(1))

torch.Size([60000, 28, 28])
torch.Size([60000, 1, 28, 28])


In [52]:
vis.image(combined_image)

'window_36297ab6a1a5f0'

In [57]:
# Better way to visualize how good our model does
# For each target value (0-9), see how often our model gets it right
def confusion(model, n, dataset):
    conf = torch.zeros(n, n)
    model.eval() # Put model in evaluation mode
    for data, target in dataset:
        data = data.unsqueeze(1)
        output = model(data)
        _, pred = torch.max(output.data, 1)
        conf[target][pred[0]] += 1
        
    # Normalize
    for i in range(n):
        conf[i] = conf[i] / conf[i].sum()
        
    vis.image(conf)
    
# A perfect model should show a diagonal white line

In [59]:
import torch.nn as nn
import torch.nn.functional as F

In [66]:
# Define our own CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(20 * 4 * 4, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        # we can also use functional interface for Conv2d and pass in the weights and biases, useful in weight sharing scenario
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x)))
        x = x.view(-1, 20 * 4 * 4) # -1 means we only care about the size of the last dimension, and just merge the rest of the dimensions into one
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)